# Azure ML on AKS with Infiniband: End-to-End Guide
---
**Table of Contents**
1. [Introduction](#1-introduction)
2. [Admin Machine Setup](#2-admin-machine-setup)
3. [Install Required Tools](#3-install-required-tools)
4. [Set Environment Variables](#4-set-environment-variables)
5. [Azure Login](#5-azure-login)
6. [Install Helm and Kubectl](#6-install-helm-and-kubectl)
7. [Create AKS Cluster and Worker Nodes](#7-create-aks-cluster-and-worker-nodes)
8. [Install Hardware Management Components](#8-install-hardware-management-components)
9. [Attach AKS to AzureML](#9-attach-aks-to-azureml)
10. [Validate Infiniband Performance](#10-validate-infiniband-performance)

---


## 1. Introduction

This tutorial will take you through creating an AML (Azure Machine Learning) compute backed by an AKS (Azure Kubernetes Service) cluster and running a distributed training job on it with Infiniband support.

The setup for this involves setup across 3 main components:
- **Admin machine**
- **Azure resources**
- **Kubernetes cluster**

Here is a diagram showing the high level setup architecture:

<img src="./assets/arch.png" width="1000">

The Admin machine component is the computer you will use to run all the commands and prepare the Kubernetes cluster for distributed training. This must be a Linux-based machine.

## 2. Admin Machine Setup

You can use an Azure ML Compute Instance to follow this tutorial, although any Linux machine will do.

We will use the AML compute instance as the base. Create a general purpose compute with 2-4 cores and 8-16 GB of RAM. Enable SSH access, root access, and add your public key.

<img src="./assets/admin-mac.png" width="1000">

Connect to it using VS Code and copy this notebook onto the Compute Instance and open it. This will simplify the setup a lot.

<img src="./assets/vscode-connect.png" width="1000">

Next, we will manage az CLI component versions and upgrades. The next section will also install az CLI if not present.

# 3. Install Required Tools

In [ ]:
%%bash
# This cell checks for Azure CLI and installs it if missing, then installs required extensions.

if ! command -v az &> /dev/null; then
    echo "Azure CLI not found. Installing Azure CLI..."
    curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash
    echo "Azure CLI installation completed."
else
    echo "Azure CLI is already installed. Version: $(az version --query '"azure-cli"' -o tsv)"
fi

# Remove any old extensions to avoid conflicts
sudo az extension remove -n azure-cli-ml
sudo az extension remove -n ml

# Add required extensions
az extension add -n ml
az extension add -n k8s-extension
az extension add --name aks-preview --allow-preview true

## 4. Set Environment Variables

Set the following environment variables for your Azure subscription, resource group, workspace, and AKS cluster configuration.

In [ ]:
# Set your Azure and AKS configuration here
import os

# Fill in your Azure subscription and resource details
os.environ["SUBSCRIPTION_ID"] = ""  # Azure Subscription ID
os.environ["RESOURCE_GROUP"] = ""   # Resource Group Name
os.environ["WORKSPACE"] = ""        # AzureML Workspace Name
os.environ["LOCATION"] = ""         # e.g., eastus, westus2, etc.
os.environ["PPG"] = "PPGNDH100"     # Proximity Placement Group name
os.environ["AKS_CLUSTER"] = "NDh100" # AKS Cluster name (will be created)
os.environ["AKS_NODE_COUNT"] = "2"  # Number of nodes in the AKS cluster
os.environ["AKS_NODE_VM_SIZE"] = "standard_nd96isr_h100_v5"  # VM size for the AKS nodes

## 5. Azure Login

Open a terminal on your machine and run `az login` before continuing. \
Login is an interactive process and requires input for the user tenant and subscription selection.

In [ ]:
%%bash
# This cell ensures you do not accidentally run the entire notebook without logging in.
# SKIP RUNNING THIS SECTION AFTER YOU LOG IN IN THE PREVIOUS STEP.
az login --identity
exit 1

In [ ]:
%%bash
# Set your Azure subscription and register required features

az account set --subscription $SUBSCRIPTION_ID
az configure --defaults workspace=$WORKSPACE group=$RESOURCE_GROUP location=$LOCATION
az feature register --name AKSInfinibandSupport --namespace Microsoft.ContainerService
az provider register -n Microsoft.ContainerService

## 6. Install Helm and Kubectl

Helm is a package manager for Kubernetes, and we will use it to install Nvidia components. \
Kubectl is a tool to operate Kubernetes clusters through CLI.

In [ ]:
%%bash
# Install kubectl and helm, add Nvidia Helm repo

sudo snap install kubectl --classic
curl -fsSL -o get_helm.sh https://raw.githubusercontent.com/helm/helm/master/scripts/get-helm-3 \
    && chmod 700 get_helm.sh \
    && ./get_helm.sh
rm get_helm.sh
helm repo add nvidia https://helm.ngc.nvidia.com/nvidia
helm repo update

## 7. Create AKS Cluster and Worker Nodes

We need the PPG setting to ensure that the nodes are colocated in the data center and the interconnect is IB. \
This reduces fault tolerance and chances of GPU allocation. \
For Hot SKUs the changes of getting allocation are low, if the allocation fails repeat without PPG. The impact will be slower IB.

In [ ]:
%%bash
# Create Proximity Placement Group (PPG) for AKS nodes

az ppg create \
--name $PPG \
--resource-group $RESOURCE_GROUP \
--location $LOCATION \
--intent-vm-sizes $AKS_NODE_VM_SIZE \
--zone 1

In [ ]:
%%bash
# Create AKS cluster

export PPG_ID=$(az ppg show -n $PPG --query 'id' -o tsv)

az aks create \
--name $AKS_CLUSTER \
--node-vm-size Standard_D8s_v3 \
--location $LOCATION \
--node-count 1 \
--nodepool-name masterpool \
--enable-managed-identity \
--generate-ssh-keys # \
# --ppg $PPG_ID # Uncomment this and above line to add PPG


In [ ]:
%%bash
# Add worker node pool with IB support

az aks nodepool add \
--cluster-name $AKS_CLUSTER \
--node-vm-size $AKS_NODE_VM_SIZE  \
--gpu-driver none \
--nodepool-name workerpool \
--enable-node-public-ip \
--node-count $AKS_NODE_COUNT

In [ ]:
%%bash
# Get AKS credentials (can be run again if needed)

az aks get-credentials \
--resource-group $RESOURCE_GROUP \
--name $AKS_CLUSTER

## 8. Install Hardware Management Components

This section installs:
1. **Node Feature Discovery:** Dynamically detects and tags available hardware on current and new nodes.
2. **Network Operator:** Installs Mellanox NIC driver and exposes them to Kubernetes. Also configures routing rules.
3. **GPU Operator:** Installs the drivers for Nvidia GPU and exposes them to Kubernetes.

This requires configuration files for Helm charts. These should have come with this tutorial; please add them in the same folder as this notebook.

We will also install the [Azure Machine Learning Kubernetes Extension](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-deploy-kubernetes-extension?view=azureml-api-2&tabs=deploy-extension-with-cli). This installs components on Kubernetes which allow AzureML to use it as an attached compute.

In [ ]:
%%bash
# Install Node Feature Discovery

helm install node-feature-discovery node-feature-discovery \
--create-namespace \
--namespace nfd-ns \
--repo https://kubernetes-sigs.github.io/node-feature-discovery/charts \
--version v0.15.6 \
--values nfd-helm.yaml \
--wait

In [ ]:
%%bash
# Install Network Operator,

helm install nvidia/network-operator \
--version 24.10.0 \
--generate-name \
--create-namespace \
--namespace no-ns \
--values nfd-helm.yaml \
--wait
kubectl apply -f nic-cluster-policy.yaml


In [ ]:
%%bash
# GPU Operator

# nvidia driver version 570
# kernel version: 5.15.0-1092-azure
helm install nvidia/gpu-operator \
--version 25.3.4 \
--generate-name \
--create-namespace \
--namespace gpu-ns \
--values gpu-operator-helm.yaml \
--wait



In [ ]:
%%bash
# AML K8s Extension

az k8s-extension create \
--name aml-k8s-extension \
--extension-type Microsoft.AzureML.Kubernetes \
--config enableTraining=True \
--cluster-type managedClusters \
--cluster-name $AKS_CLUSTER \
--scope cluster \
--auto-upgrade false \
--release-train staging \
--version 1.1.146 # This version has IB support

## 9. Attach AKS to AzureML

Now the cluster is ready, we will attach it to AzureML to use as a compute.

For Kubernetes computes, we must define the [instance type](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-manage-kubernetes-instance-types?view=azureml-api-2&tabs=select-instancetype-to-trainingjob-with-cli%2Cselect-instancetype-to-modeldeployment-with-cli%2Cdefine-resource-to-modeldeployment-with-cli), which represents the specs of a single node of your training job. We will create an instance type which consumes all resources of 1 worker node.

In [ ]:
%%bash
# Attach AKS cluster to AzureML and apply instance type

az ml compute attach \
--resource-group $RESOURCE_GROUP \
--workspace-name $WORKSPACE \
--type Kubernetes \
--name "kube-compute" \
--resource-id "/subscriptions/$SUBSCRIPTION_ID/resourceGroups/$RESOURCE_GROUP/providers/Microsoft.ContainerService/managedclusters/$AKS_CLUSTER" \
--identity-type SystemAssigned

kubectl apply -f instance_type.yaml

## 10. Validate Infiniband Performance

The setup is now complete. AKS training is an existing feature of AML; this article is about performant training through Infiniband.

With that in mind, we will run the [nccl-test](https://github.com/NVIDIA/nccl) to validate Infiniband performance. All-reduce under nccl-test is a job which causes large data transfer from each node to all other nodes; we measure the bus speed under the stress of this job.

The bus speed is a number representing the performance of the network, agnostic of the number of nodes (i.e., GPUs/ranks) used in the test.

<img src="./assets/all_reduce.png" width=600>

To run this test, we will first build an AzureML environment with a suitable docker image and nccl-test installed. Environment creation should take about 30 mins to complete, then we will run a job in that environment.

In [ ]:
%%bash
# Create AzureML environment for NCCL test

az ml environment create -f env.yaml -g $RESOURCE_GROUP -w $WORKSPACE

Wait until the environment creation is complete. The next step will fail if the environment is not ready.

In [ ]:
%%bash
# Run NCCL test job in AzureML

az ml environment show -n nccltest_wrapper --version 1 > /dev/null
az ml job create -f job.yaml

### Expected Results

You should expect to see a bus bandwidth of around **147** in the output of the job in process_0 logs. Note that we take the maximum speed achieved as indicator of hardware speed, as this means the algorithm has managed to load the hardware appropriately at those settings.

Avg bus bandwidth    : 147.439

This confirms that IB is usable on the cluster with AML, and has optimum performance.